In [1]:
#Generating Text with an LSTM Model

In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
 
# load ascii text and covert to lowercase
filename = "/home/nmit/Downloads/wonderland/wonder/wonderland.txt"
raw_text = open(filename, 'r', encoding='utf-8').read()
raw_text = raw_text.lower()
 
# create mapping of unique chars to integers
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
 
# summarize the loaded data
n_chars = len(raw_text)
n_vocab = len(chars)
print("Total Characters: ", n_chars)
print("Total Vocab: ", n_vocab)
 
# prepare the dataset of input to output pairs encoded as integers
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
    seq_in = raw_text[i:i + seq_length]
    seq_out = raw_text[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print("Total Patterns: ", n_patterns)
 
# reshape X to be [samples, time steps, features]
X = torch.tensor(dataX, dtype=torch.float32).reshape(n_patterns, seq_length, 1)
X = X / float(n_vocab)
y = torch.tensor(dataY)
 
class CharModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.lstm = nn.LSTM(input_size=1, hidden_size=256, num_layers=1, batch_first=True)
        self.dropout = nn.Dropout(0.2)
        self.linear = nn.Linear(256, n_vocab)
    def forward(self, x):
        x, _ = self.lstm(x)
        # take only the last output
        x = x[:, -1, :]
        # produce output
        x = self.linear(self.dropout(x))
        return x
    
n_epochs = 40
batch_size = 128
model = CharModel()
 
optimizer = optim.Adam(model.parameters())
loss_fn = nn.CrossEntropyLoss(reduction="sum")
loader = data.DataLoader(data.TensorDataset(X, y), shuffle=True, batch_size=batch_size)
 
best_model = None
best_loss = np.inf
for epoch in range(n_epochs):
    model.train()
    for X_batch, y_batch in loader:
        y_pred = model(X_batch)
        loss = loss_fn(y_pred, y_batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    # Validation
    model.eval()
    loss = 0
    with torch.no_grad():
        for X_batch, y_batch in loader:
            y_pred = model(X_batch)
            loss += loss_fn(y_pred, y_batch)
        if loss < best_loss:
            best_loss = loss
            best_model = model.state_dict()
        print("Epoch %d: Cross-entropy: %.4f" % (epoch, loss))
 
torch.save([best_model, char_to_int], "/home/nmit/Documents/single-char.pth")

Total Characters:  13637
Total Vocab:  41
Total Patterns:  13537
Epoch 0: Cross-entropy: 39879.5938
Epoch 1: Cross-entropy: 39814.8555
Epoch 2: Cross-entropy: 39797.9297
Epoch 3: Cross-entropy: 39622.3633
Epoch 4: Cross-entropy: 38937.0547
Epoch 5: Cross-entropy: 38132.8203
Epoch 6: Cross-entropy: 37738.0234
Epoch 7: Cross-entropy: 37107.9922
Epoch 8: Cross-entropy: 36685.9766
Epoch 9: Cross-entropy: 36411.8008
Epoch 10: Cross-entropy: 36269.5391
Epoch 11: Cross-entropy: 35955.3203
Epoch 12: Cross-entropy: 35638.4023
Epoch 13: Cross-entropy: 35374.9375
Epoch 14: Cross-entropy: 35158.4102
Epoch 15: Cross-entropy: 35041.2148
Epoch 16: Cross-entropy: 34733.2344
Epoch 17: Cross-entropy: 34407.3008
Epoch 18: Cross-entropy: 34150.3164
Epoch 19: Cross-entropy: 33780.7070
Epoch 20: Cross-entropy: 33540.3789
Epoch 21: Cross-entropy: 33197.9297
Epoch 22: Cross-entropy: 32690.6680
Epoch 23: Cross-entropy: 32208.5352
Epoch 24: Cross-entropy: 31652.2441
Epoch 25: Cross-entropy: 31191.2051
Epoch 26:

In [10]:
#Generating Text with an LSTM Model

In [11]:
seq_length = 100
start = np.random.randint(0, len(raw_text)-seq_length)
prompt = raw_text[start:start+seq_length]

In [12]:
import numpy as np
import torch
import torch.nn as nn
 
best_model, char_to_int = torch.load("/home/nmit/Documents/single-char.pth")
n_vocab = len(char_to_int)
int_to_char = dict((i, c) for c, i in char_to_int.items())
 
# reload the model
class CharModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.lstm = nn.LSTM(input_size=1, hidden_size=256, num_layers=1, batch_first=True)
        self.dropout = nn.Dropout(0.2)
        self.linear = nn.Linear(256, n_vocab)
    def forward(self, x):
        x, _ = self.lstm(x)
        # take only the last output
        x = x[:, -1, :]
        # produce output
        x = self.linear(self.dropout(x))
        return x
model = CharModel()
model.load_state_dict(best_model)
 
# randomly generate a prompt
filename = "/home/nmit/Downloads/wonderland/wonder/wonderland.txt"
seq_length = 100
raw_text = open(filename, 'r', encoding='utf-8').read()
raw_text = raw_text.lower()
start = np.random.randint(0, len(raw_text)-seq_length)
prompt = raw_text[start:start+seq_length]
pattern = [char_to_int[c] for c in prompt]
 
model.eval()
print('Prompt: "%s"' % prompt)
with torch.no_grad():
    for i in range(1000):
        # format input array of int into PyTorch tensor
        x = np.reshape(pattern, (1, len(pattern), 1)) / float(n_vocab)
        x = torch.tensor(x, dtype=torch.float32)
        # generate logits as output from the model
        prediction = model(x)
        # convert logits into one character
        index = int(prediction.argmax())
        result = int_to_char[index]
        print(result, end="")
        # append the new character into the prompt for the next iteration
        pattern.append(index)
        pattern = pattern[1:]
print()
print("Done.")

Prompt: "a crimson velvet cushion; and, last of all this grand procession, came the king and queen of hearts."
 al ce caca  oo d sai quecl  and toild she rerte fand nn rorel  and aoong the  ai ce co cocwe then  alice could het remd no the cickes so the kaoe  fn  al the woned  aid she qait ho woete with the kicg torele an cecces, ait w shed toe dingh af rares to tee cousdon, and the hreput onee tu toiel whsh oetele ao ce coued ih  alice toite rabbit, oitt an  alice toit  and woin  uu dor tonte the rerte fand re merele aid e moip tu toese oith oo mirele aii a serg lont so tir luoking oo rert lomtle  “iu aan the wan  and soon so tee sooer  and toon do she wonte aarer hnd tee iocpusn aese here the greaess  and woon woue dand nn rer huoning oo rer luoting aoint te tee sooer sare colnnes, “hot at she wast head  “ih  an  who’ loat the rabt to tes tooklng an the dins  and a cang lo cis hod ofoee th aed tet eoonnng at ave whe gerre aed t eadi  oo the cac n vernee ao cacrre  fl cas tes rorele an